In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pickle
from utilities import *
from DCS import *
from sentences import *
import pandas as pd
import numpy as np

In [3]:
# goodDict = pickle.load(open('mergedGood_v3.p', 'rb'))


In [4]:
# fList = list(goodDict.keys())


In [5]:
# SeeDCS(dcsO)

In [6]:
# skt, dcs  = loadSentence(fList[7], goodDict[fList[7]])
# SeeSentence(skt)

In [3]:
df_pf1 = pd.read_csv('extras/pvbnhi@pf.csv', header=None, names=['file', 'miss', 'miss2'])
df_pf2 = pd.read_csv('extras/pvbnhi@skt.csv', header=None, names=['file', 'miss', 'miss2'])
df_pf3 = pd.read_csv('extras/pvbnhi@upd.csv', header=None, names=['file', 'miss', 'miss2'])

# df_pf1['miss'] = df_pf1[df_pf1['miss'] != 'Incomplete chunk']['miss'].astype(int)
df_pf1 = df_pf1[df_pf1['miss'] == '0']
df_pf1['folder'] = '../TextSegmentation/Pickle_Files/'

# df_pf2['miss'] = df_pf2[df_pf2['miss'] != 'Incomplete chunk']['miss'].astype(int)
df_pf2 = df_pf2[df_pf2['miss'] == '0']
df_pf2['folder'] = '../TextSegmentation/corrected_10to20/'

# df_pf3['miss'] = df_pf3[df_pf3['miss'] != 'Incomplete chunk']['miss'].astype(int)
df_pf3 = df_pf3[df_pf3['miss'] == '0']
df_pf3['folder'] = '../TextSegmentation/Updated Pickles/'

frames = [df_pf1, df_pf2, df_pf3]
df_pf = pd.concat(frames)
%reset_selective df_pf1
%reset_selective df_pf2
%reset_selective df_pf3

goodFileDict = {}
for index, row in df_pf.iterrows():
    goodFileDict['%d.p' % row['file']] = '%s%d.p' % (row['folder'], row['file'])
len(goodFileDict)

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y
Once deleted, variables cannot be recovered. Proceed (y/[n])?  y
Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


56974

In [25]:
fList = list(goodFileDict.keys())

def ValidateSandhi(fi = -1, fName = '', fPath='', verbose=False):
    
    if(fi >= 0):
        f = fList[fi]
        fPath = goodFileDict[f]
        skt, dcs  = loadSentence(f, fPath)
    else:
        f = fName
        skt, dcs  = loadSentence(fName, fPath)
    if skt==None:
        return
    if verbose:
        print()
        print('=='*20)
        print(f.upper())
        print(fPath)
#     SeeSentence(skt)
    (chunkDict, lemmaList, wordList, revMap2Chunk, qu, cngList, verbs, tuplesMain) = SentencePreprocess(skt)
    if(len(chunkDict) != len(dcs.lemmas)):
        print('*'*66)
        print('REPORTING LENGTH MISMATCH:', fPath)
        print('*'*66)
        return 0
    toSearch = []

    for i in range(len(dcs.lemmas)):
        lls = dcs.lemmas[i]
        for j in range(len(lls)):
            # (chunk, lemma, cng)
            toSearch.append((i, rom_slp(lls[j]), int(dcs.cng[i][j])))

    deactTuple = [False]*len(tuplesMain)
    for qtup in toSearch:
        if verbose:
            print('\n\n[QUERY]', qtup)
        qcid = qtup[0]
        qlem = qtup[1]
        qcng = qtup[2]
        activeChunk = chunkDict[qcid]
        matchFound = False

        for pos in activeChunk.keys():
            for i in activeChunk[pos]:
                if not deactTuple[i]:
                    for tup in tuplesMain[i]:
        #                 print(tup)
                        if (tup[2] == qtup[1]) and (tup[3] == qtup[2]):
                            if(verbose):
                                print('[PAIR Match] chunk_%d, pos_%d, [%s], cng(%d)' % (qcid, pos, tup[2], tup[3]))
                            matchFound = True
                            deactTuple[i] = True
                            srch = (pos, i)
                            break
                if(matchFound):
                    break
            if(matchFound):
                break

        if not matchFound:
            for pos in activeChunk.keys():
                for i in activeChunk[pos]:
                    if not deactTuple[i]:
                        for tup in tuplesMain[i]:
                            if tup[2] == qtup[1]:
                                if(verbose):
                                    print('[LEMMA Match] chunk_%d, pos_%d, [%s], cng(%d)' % (qcid, pos, tup[2], tup[3]))
                                matchFound = True
                                deactTuple[i] = True
                                srch = (pos, i)
                                break
                    if(matchFound):
                        break
                if(matchFound):
                    break

        if matchFound:
            if verbose:
                print('[REMOVED] (Pos, Lemma, Names)')
#             print(srch)
            n1 = tuplesMain[srch[1]][0][1]
#             print(n1)
            p1 = srch[0]
            for pos in activeChunk.keys():
                if(pos == srch[0]):
                    for i in activeChunk[pos]:
                        # Remove all
                        deactTuple[i] = True
                        if verbose:
                            for t_rem in tuplesMain[i]:
                                print('(pos_%d, [%s], [%s]) ' % (pos, t_rem[2], t_rem[1]))
                else:
                    if(pos < p1):
                        for i in activeChunk[pos]:
                            # Deactivate Tuple
                            if not deactTuple[i]:
                                n2 = tuplesMain[i][0][2]
                                if not CanCoExist_sandhi(pos, p1, n2, n1):
                                    deactTuple[i] = True
                                    if(verbose):
                                        for t_rem in tuplesMain[i]:
                                            print('(pos_%d, [%s], [%s]) ' % (pos, t_rem[2], t_rem[1]))
                    else:
                        for i in activeChunk[pos]:
                            # Deactivate Tuple
                            if not deactTuple[i]:
                                n2 = tuplesMain[i][0][2]
                                if not CanCoExist_sandhi(p1, pos, n1, n2):
                                    deactTuple[i] = True
                                    if(verbose):
                                        for t_rem in tuplesMain[i]:
                                            print('(pos_%d, [%s], [%s]) ' % (pos, t_rem[2], t_rem[1]))
        else:
            if verbose:
                print('___[NOT FOUND]___')
    if verbose:
        print('\n[REMAINING] (cid, pos, tuples)')
        for cid in list(chunkDict.keys()):
            for pos in chunkDict[cid].keys():
                for tid in chunkDict[cid][pos]:
                    if not deactTuple[tid]:
                        print('\n', [cid, pos], '\t', end='')
                        for tup in tuplesMain[tid]:
                            print(tup[2], end=', ')
    if verbose:
        print('\n\nWords remaining in skt[%s]:' % f, len(tuplesMain) - sum(deactTuple))
    return len(tuplesMain) - sum(deactTuple)

In [23]:
# ValidateSandhi(fName = '305755.p', fPath= '../TextSegmentation/Pickle_Files/305755.p', verbose=True)
ValidateSandhi(2, verbose=True)


8494.P
../TextSegmentation/Updated Pickles/8494.p


[QUERY] (0, 'DAnya', 1)
[LEMMA Match] chunk_0, pos_0, [DAnya], cng(3)
[REMOVED] (Pos, Lemma, Names)
(pos_0, [DAnya], [DAnya]) 
(pos_0, [DAna], [DAnI]) 
(pos_0, [DAna], [DAnI]) 
(pos_4, [f], [ara]) 
(pos_4, [f], [ara]) 
(pos_4, [f], [ara]) 


[QUERY] (0, 'rASi', 1)
[LEMMA Match] chunk_0, pos_5, [rASi], cng(49)
[REMOVED] (Pos, Lemma, Names)
(pos_4, [ara], [ara]) 
(pos_4, [ara], [arA]) 
(pos_5, [rASi], [rASO]) 
(pos_5, [rA], [rA]) 
(pos_5, [rA], [rA]) 
(pos_6, [ASa], [ASO]) 
(pos_6, [ASa], [ASO]) 
(pos_6, [ASu], [ASO]) 
(pos_6, [ASu], [ASO]) 
(pos_6, [ASa], [ASO]) 


[QUERY] (1, 'sTA', -190)
[PAIR Match] chunk_1, pos_0, [sTA], cng(-190)
[REMOVED] (Pos, Lemma, Names)
(pos_0, [sTta], [sTitam]) 
(pos_0, [sTta], [sTitam]) 
(pos_0, [sTta], [sTitam]) 
(pos_0, [sTA], [sTitam]) 


[QUERY] (2, 'mAsa', 1)
[LEMMA Match] chunk_2, pos_0, [mAsa], cng(31)
[REMOVED] (Pos, Lemma, Names)
(pos_0, [mAsa], [mAsam]) 
(pos_0, [mAsa], [mAsam]) 
(pos_0, [mAsa],

1

In [ ]:
got = 0
for i in range(500):
    r = ValidateSandhi(i, verbose= False)
#     print('Word Remaining:', r)
    if r > 0:
        got += 1
        ValidateSandhi(i, verbose=True)
    if got >= 100:
        break
print('-'*50, '\nGot', got, 'problems')

In [23]:
type(goodFileDict) ==dict

True

In [63]:
goodFileDict[8454]

KeyError: 8454